# Task 1: implementation

In [18]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re


1. Websites that have the largest count of news articles

In [21]:

# Load the data
data_df = pd.read_csv('C:\\Users\\araso\\Documents\\10academy\\data\\data.csv')
domains_location_df = pd.read_csv('C:\\Users\\araso\\Documents\\10academy\\data\\domains_location.csv')
traffic_data_df = pd.read_csv('C:\\Users\\araso\\Documents\\10academy\\data\\traffic.csv')
rating_df = pd.read_csv('C:\\Users\\araso\\Documents\\10academy\\data\\rating.csv')

# Merge data.csv with domains_location.csv
data_with_location = pd.merge(data_df, domains_location_df, left_on='source_name', right_on='SourceCommonName', how='left')

# Merge the result with traffic_data.csv
full_data = pd.merge(data_with_location, traffic_data_df, left_on='SourceCommonName', right_on='Domain', how='left')

# 1. Websites that have the largest count of news articles
article_counts = data_df['source_name'].value_counts()
top_10_websites_articles = article_counts.head(10)
bottom_10_websites_articles = article_counts.tail(10)

# Display the results
print("Top 10 Websites by Article Count:")
print(top_10_websites_articles)
print("\nBottom 10 Websites by Article Count:")
print(bottom_10_websites_articles)




Top 10 Websites by Article Count:
source_name
ETF Daily News        16631
The Times of India     7629
GlobeNewswire          5485
Forbes                 4423
Biztoc.com             3968
BBC News               3342
Globalsecurity.org     3093
Business Insider       2746
ABC News               2188
Marketscreener.com     1948
Name: count, dtype: int64

Bottom 10 Websites by Article Count:
source_name
Powerdns.com           1
Frame.work             1
Dannymekic.com         1
Dougwils.com           1
Linuxgizmos.com        1
Paranatural.net        1
Duringquiettime.com    1
Askmen.com             1
Elementy.ru            1
Caltech.edu            1
Name: count, dtype: int64


2. Websites with the highest numbers of visitors traffic

In [22]:
# 2. Websites with the highest numbers of visitors traffic
top_10_websites_traffic = traffic_data_df.sort_values(by='GlobalRank').head(10)[['Domain', 'GlobalRank']]
bottom_10_websites_traffic = traffic_data_df.sort_values(by='GlobalRank').tail(10)[['Domain', 'GlobalRank']]

print("\nTop 10 Websites by Visitor Traffic:")
print(top_10_websites_traffic)
print("\nBottom 10 Websites by Visitor Traffic:")
print(bottom_10_websites_traffic)


Top 10 Websites by Visitor Traffic:
                 Domain  GlobalRank
0            google.com           1
1          facebook.com           2
2           youtube.com           3
3           twitter.com           4
4         instagram.com           5
5          linkedin.com           6
6             apple.com           7
7         microsoft.com           8
8  googletagmanager.com           9
9         wikipedia.org          10

Bottom 10 Websites by Visitor Traffic:
                        Domain  GlobalRank
999990            eiretrip.com      999991
999991    exploring-africa.com      999992
999992                hmag.com      999993
999993          irishcycle.com      999994
999994         keith-baker.com      999995
999995              kireie.com      999996
999996             mt-lock.com      999997
999997           pinkwater.com      999998
999998          soderhomes.com      999999
999999  toyotamusicfactory.com     1000000


3. Countries with the highest number of news media organisations (represented by domains in the data)

In [23]:
# 3. Countries with the highest number of news media organisations (represented by domains in the data)
country_domain_counts = domains_location_df['Country'].value_counts()
top_10_countries_domains = country_domain_counts.head(10)
bottom_10_countries_domains = country_domain_counts.tail(10)

print("\nTop 10 Countries by Number of Media Organisations:")
print(top_10_countries_domains)
print("\nBottom 10 Countries by Number of Media Organisations:")
print(bottom_10_countries_domains)


Top 10 Countries by Number of Media Organisations:
Country
United States     14111
United Kingdom     1950
Italy              1810
France             1041
Russia             1024
Canada              887
Germany             884
China               780
Turkey              725
India               686
Name: count, dtype: int64

Bottom 10 Countries by Number of Media Organisations:
Country
American Samoa              1
Micronesia                  1
Mayotte                     1
Isle of Man                 1
Guernsey                    1
West Bank                   1
Pitcairn Islands            1
Oceans                      1
Turks and Caicos Islands    1
Saint Helena                1
Name: count, dtype: int64


4. Countries with Many Articles Written About Them

In [36]:
# 4. Countries that have many articles written about them
# We will check if the country names are mentioned in the article content

# List of countries to check in the content
countries_list = domains_location_df['Country'].unique()

# Initialize a dictionary to count articles per country
country_article_count = {country: 0 for country in countries_list}

# Count occurrences of each country in the article content
for country in countries_list:
    country = str(country)
    country_article_count[country] = data_df['content'].str.contains(country, case=False, na=False).sum()

# Convert dictionary to DataFrame
country_article_count_df = pd.DataFrame(list(country_article_count.items()), columns=['Country', 'ArticleCount']).dropna()

# Sort by ArticleCount to find the top and bottom 10 countries
top_10_countries_articles = country_article_count_df.sort_values(by='ArticleCount', ascending=False).head(10)
bottom_10_countries_articles = country_article_count_df.sort_values(by='ArticleCount', ascending=True).head(10)

# Display the results
print("Top 10 Countries by Article Count:")
print(top_10_countries_articles)

print("\nBottom 10 Countries by Article Count:")
print(bottom_10_countries_articles)

{'Senegal': np.int64(52), 'France': np.int64(600), 'Italy': np.int64(355), 'Argentina': np.int64(222), 'China': np.int64(1773), 'Germany': np.int64(467), 'Ukraine': np.int64(1289), 'United States': np.int64(1614), 'Russia': np.int64(2089), 'Egypt': np.int64(295), 'India': np.int64(3746), 'Taiwan': np.int64(241), 'United Kingdom': np.int64(190), 'Canada': np.int64(868), 'South Africa': np.int64(540), 'Greece': np.int64(144), 'Netherlands': np.int64(249), 'Chile': np.int64(90), 'Iran': np.int64(1079), 'Cuba': np.int64(114), 'Lithuania': np.int64(113), 'Hungary': np.int64(134), 'Uruguay': np.int64(66), 'Brazil': np.int64(309), 'Switzerland': np.int64(159), 'Armenia': np.int64(224), 'Indonesia': np.int64(282), 'Azerbaijan': np.int64(191), 'Bangladesh': np.int64(185), 'Georgia': np.int64(275), 'Tuvalu': np.int64(10), 'Venezuela': np.int64(131), 'Spain': np.int64(364), 'Latvia': np.int64(78), 'Cameroon': np.int64(58), 'Mexico': np.int64(392), 'United Arab Emirates': np.int64(152), 'Portugal'

5. Websites that reported about Africa, US, China, EU, Russia, Ukraine, Middle East

In [37]:
# 5. Websites that reported about Africa, US, China, EU, Russia, Ukraine, Middle East

# Define country groups for each region
african_countries = domains_location_df[domains_location_df['location'].isin(['DZ', 'AO', 'BJ', 'BW', 'BF', 'BI', 'CV', 'CM', 'CF', 'TD', 'KM', 'CG', 'CD', 'DJ', 'EG', 'GQ', 'ER', 'SZ', 'ET', 'GA', 'GM', 'GH', 'GN', 'GW', 'KE', 'LS', 'LR', 'LY', 'MG', 'MW', 'ML', 'MR', 'MU', 'YT', 'MA', 'MZ', 'NA', 'NE', 'NG', 'RE', 'RW', 'SH', 'ST', 'SN', 'SC', 'SL', 'SO', 'ZA', 'SS', 'SD', 'TZ', 'TG', 'TN', 'UG', 'EH', 'ZM', 'ZW'])]['SourceCommonName'].unique()

eu_countries = domains_location_df[domains_location_df['location'].isin(['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'])]['SourceCommonName'].unique()

middle_east_countries = domains_location_df[domains_location_df['location'].isin(['BH', 'EG', 'IR', 'IQ', 'IL', 'JO', 'KW', 'LB', 'OM', 'PS', 'QA', 'SA', 'SY', 'AE', 'YE'])]['SourceCommonName'].unique()

# Initialize dictionaries for each region
africa_article_count = 0
us_article_count = 0
china_article_count = 0
eu_article_count = 0
russia_article_count = 0
ukraine_article_count = 0
middle_east_article_count = 0

# Check occurrences for each region in the article content
africa_article_count = data_df['content'].apply(lambda x: any(country in x for country in african_countries)).sum()
us_article_count = data_df['content'].str.contains('US', case=False, na=False).sum()
china_article_count = data_df['content'].str.contains('China', case=False, na=False).sum()
eu_article_count = data_df['content'].apply(lambda x: any(country in x for country in eu_countries)).sum()
russia_article_count = data_df['content'].str.contains('Russia', case=False, na=False).sum()
ukraine_article_count = data_df['content'].str.contains('Ukraine', case=False, na=False).sum()
middle_east_article_count = data_df['content'].apply(lambda x: any(country in x for country in middle_east_countries)).sum()

# Aggregate the article counts for each region
region_article_count = {
    'Africa': africa_article_count,
    'US': us_article_count,
    'China': china_article_count,
    'EU': eu_article_count,
    'Russia': russia_article_count,
    'Ukraine': ukraine_article_count,
    'Middle East': middle_east_article_count
}

region_article_count_df = pd.DataFrame(list(region_article_count.items()), columns=['Region', 'ArticleCount'])

# Sort and display the top and bottom 10 websites for these regions
top_10_region_articles = region_article_count_df.sort_values(by='ArticleCount', ascending=False).head(10)
bottom_10_region_articles = region_article_count_df.sort_values(by='ArticleCount', ascending=True).head(10)


print("\nTop 10 Regions by Article Count:")
print(top_10_region_articles)

print("\nBottom 10 Regions by Article Count:")
print(bottom_10_region_articles)



In [34]:
# 6. Websites with the Highest Count of Sentiments
# Group data by source_name and calculate descriptive statistics
sentiment_stats = rating_df.groupby('source_name')['title_sentiment'].agg(['mean', 'median', 'var', 'count'])

# Top 10 domains by visitor traffic
top_10_domains = traffic_data_df.sort_values(by='GlobalRank').head(10)['Domain']

# Sentiment distribution for top 10 domains by traffic
top_10_sentiment_distribution = rating_df[rating_df['source_name'].isin(top_10_domains)].groupby('source_name')['title_sentiment'].describe()

# Global sentiment distribution
global_sentiment_distribution = rating_df['title_sentiment'].describe()

print("\nWebsites with the Highest Count of Sentiments (Mean, Median, Variance):")
print(sentiment_stats.sort_values(by='count', ascending=False).head(10))

print("\nSentiment Distribution for Top 10 Domains by Traffic:")
print(top_10_sentiment_distribution)

print("\nGlobal Sentiment Distribution:")
print(global_sentiment_distribution)

KeyboardInterrupt: 